## Jupyter Notebook space for Passive Active NLP StageLens project

General module loading

In [1]:
!python --version
!python -m spacy download en_core_web_md

print("Downloaded")

Python 3.6.3
    100% |████████████████████████████████| 120.9MB 18.7MB/s ta 0:00:01   61% |███████████████████▊            | 74.4MB 4.2MB/s eta 0:00:12    82% |██████████████████████████▍     | 99.6MB 4.2MB/s eta 0:00:06
  Running setup.py install for en-core-web-md ... done

    Linking successful
    /srv/venv/lib/python3.6/site-packages/en_core_web_md -->
    /srv/venv/lib/python3.6/site-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')

Downloaded


In [6]:
import spacy
import html
from spacy import displacy
import re
PATTERN = "[^a-zA-Z0-9_\s\.']+"
rgx = re.compile(PATTERN, re.IGNORECASE)

nlp = spacy.load('en_core_web_md')
print("Loaded models")

Loaded models


Preprocessing input text

In [31]:
tsv = """Sometimes I wish that	my mom would come home more often.	|1| my mom |2| would come home more often.
My family	has a cat...I don't have a cat though I have 2 dogs	|1| has a cat...|2| I don't have a cat |3| though I have 2 dogs
When I am nervous	my hands get shaky	|1| my hands |2| get shaky
At times I worry about	my own dreams, nightmares I have thousands of them	|1| my own dreams, nightmares |2| I have thousands of them
Technology	it's crazy and i wish i could keep up	|1| its crazy and |2| i wish |3| i could keep up
When I get mad	I sometimes use big voice then I go somewhere quiet so I can cry by myself.	|1| I sometimes use big voice |2| then I go somewhere quiet |3| so I can cry by myself.
My main problem is	a feeling that I am not entitled	|1| a feeling |2| that I am not entitled
If my mother	was a different lady, I would not be who I am.	|1| was a different lady, |2| I would not be who I am.
When I get mad	it takes a long time to build up, but I am slow to forgive.	|1| it takes a long time to build up, |2| but I am slow to forgive.
When they avoided me	I felt safe, knowing that I could receive a warm welcome from my family and friends.	|1| I felt safe, |2| knowing |3| that I could receive a warm welcome |4| from my family and friends.
Sometimes I wish that	I could be younger and have done things differently.	|1| I could be younger |2| and have done things differently.
When they avoided me	I sought to find out 'why?' . Knowing the 'why' can begin Awareness and Healing	|1| I sought |2| to find out 'why?' . |3| Knowing the 'why' |4| can begin Awareness and Healing
Being with other people	is both a learning experience and an opportunity to exchange views.	|1| is both a learning experience and an opportunity |2| to exchange views.
My main problem is	only a main problem if I think it is.	|1| only a main problem |2| if I think |3| it is.
When people are helpless	At times I try to find other ways of doing thing. But sometime I might be disappointed and stop or quit.	|1| At times I try |2| to find other ways |3| of doing thing. |4| But sometime I might be disappointed |5| and stop |6| or quit.
My father	taught me how to think critically and to love deeply.	|1| taught me |2| how to think critically |3| and to love deeply.
If I can\'t get what I want	I try to work out another way of getting what I want	|1| I try |2| to work out another way |3| of getting |4| what I want
When they avoided me	hmmm... I\'m stumped. I have no idea how to respond to this... Although there is a sense of relief...	|1| hmmm... I'm stumped. |2| I have no idea |3| how to respond to this...|4| Although there is a sense of relief...
Loving other people	can deepen with familiarity. Even with those who aren't a natural fit the differences between us can add color to the world as well as an opportunity to turn inward to see what is being triggered. Having said that, it's difficult for me to open my heart and trust.	|1| can deepen with familiarity. |2| Even with those who aren't a natural fit |3| the differences between us can add color to the world |4| as well as an opportunity to turn inward |5| to see |6| what is being triggered. |7| Having said that, |8| it's difficult for me |9| to open |10| my heart |11| and trust.
When people are helpless	I am inspired to reach out and be of service.	|1| I am inspired |2| to reach out |3| and be of service.
Rules	are guard rails. It is good and helpful to have them. When you head in the right direction you never get in touch with them, no matter how fast or slow you travel. Good rules are not in your way.	|1| are guard rails. |2| It is good and helpful |3| to have them. |4| When you head in the right direction |5| you never get in touch with them, |6| no matter how fast or slow you travel. |7| Good rules are not in |8| your way.
Technology	is a great tool, nice to have and can bring global humanity closer together in ways never before imagined. But if it were to disappear tomorrow, I'm of the generation that would be able to survive.	|1| is a great tool, |2| nice to have |3| and can bring global humanity closer together |4| in ways never before imagined. |5| But if it were |6| to disappear tomorrow, |7| I'm of the generation |8| that would be able |9| to survive.
A good boss	A good boss is critical toward making a workplace environment healthy and successful!	|1| A good boss is critical |2| toward making a workplace environment healthy and successful!
My main problem is	getting over decades spent proving myself.	|1| getting over |2| decades spent proving myself.
I feel sorry	for the pain that all humans and all beings on this earth must endure as part of the experience of being here.	|1| for the pain |2| that all humans and all beings on this earth must endure |3| as part of the experience of being here.
If I were in charge	i would change that by facilitating change that made everyone in charge ... to the degree of their capabilities in wordly matters, but always in charge of their interior and their right to decide for themselves what is real and true for them.	|1| i would change that |2| by facilitating change |3| that made everyone in charge ...|4| to the degree of their capabilities in wordly matters,|5| but always in charge |6| of their interior and |7| their right |8| to decide for themselves |9| what is real and true for them.
At times I worry about	big, big existential questions, and little microscopic ones... sometimes within milliseconds of one another.	|1| big, big existential questions, and little microscopic ones...|2| sometimes within milliseconds of one another.
Raising a family	is a profound gift. I am tending the future. I am tending the life processes. It is a place for love and life and evolution to soar.	|1| is a profound gift. |2| I am tending the future. |3| I am tending the life processes. |4| It is a place for love and life and evolution |5| to soar.
At times I worry about	how differently I see and process things from many of the people I encounter, and whether I am wisely acting and interacting.	|1| how differently I see |2| and process things |3| from many of the people I encounter, |4| and whether I am wisely acting |5| and interacting.
What I like to do best is	be a conduit. There is no I.	|1| be a conduit. |2| There is no I.
Being with other people	can be an isolating or a hugely enriching experience, the latter when each others presence liberates emergent thoughts or self expressions bringing forth a collective that is more than the sum of its parts	|1| can be an isolating or a hugely enriching experience, |1| the latter when each others presence liberates emergent thoughts or self expressions |3| bringing forth a collective |4| that is more than the sum of its parts
A teacher has the right to	I don't mean to be oppositional. . but this sentence stem. . .like so many others. . isn't one I can really answer outside of context. Rights. This is a wobbly construct. I'm sorry. I'm out of time otherwise I would write a bit about the possibility of rights . .the idea of having a right. . the usage of the language of rights.	|1| I don't mean |2| to be oppositional. .|3| but this sentence stem. . .like so many others. . isn't one |4| I can really answer outside of context.|5| Rights. This is a wobbly construct.|6| I'm sorry.|7| I'm out of time |8| otherwise I would write a bit about the possibility of rights ..the idea of having a right. . the usage of the language of rights.
Being with other people	is most often a pleasure and fascinating experience of seeing a completely different version of Self.	|1| is most often a pleasure and fascinating experience |2| of seeing a completely different version of Self.
The past	is a construct to give form and structure to the present moment.	|1| is a construct |2| to give form and structure to the present moment.
We could make the world a better place if	the mystery was okay to be as a question, that is to say if the question was okay to be a question and the answer could be a question and the answers and questions could be held together because they need each other and we realize a little faster that they won't end and they are growing together and rest in that.	|1| the mystery was okay |2| to be as a question, |3| that is to say |4| if the question was okay |5| to be a question |6| and the answer could be a question |7| and the answers and questions could be held together |8| because they need each other |9| and we realize a little faster |10| that they won't end |11| and they are growing together |12| and rest in that.
A good boss	is a solar being ... one that brings light and eternal love without demanding anything of the others ... an energetic and gravitational field of such calm, peaceful magnitude that all are warmed and captivated by their glow ... brings a new burst life to all those who are in their presence	|1| is a solar being ...|2| one that brings light and eternal love |3| without demanding anything of the others ..|4| an energetic and gravitational field of such calm, peaceful magnitude |5| that all are warmed and captivated by their glow ... |6| brings a new burst life |7| to all those who are in their presence
My conscience bothers me if	I'm not placing my feet on the path with an open heart, mind and spirit, tuning in to what's moving and showing up fully at the dance.	|1| I'm not placing |2| my feet on the path with an open heart, mind and spirit,|3| tuning in to |4| what's moving |5| and showing up fully at the dance.
When I am criticized	I can embody my personality and feel a pang contrast to the story I tell myself inside the story we tell ourselves of what we think in the moment to be an absolute before it falls away into a deeper, clearer calm of letting go.	|1| I can embody |2| my personality |3| and feel a pang contrast |4| to the story I tell myself |5| inside the story we tell ourselves |6| of what we think in the moment |7| to be an absolute |8| before it falls away into a deeper, clearer calm |9| of letting go.
The thing I like about myself is	that here I sit in the drop of this essential moment, chuckling at this question! :-D	|1| that here I sit in the drop of this essential moment, |2| chuckling at this question! :-D
My main problem is	integrating emptiness with everyday life, finding meaning in meaningless given the meaningless in meaning making.	|1| integrating emptiness with everyday life, |2| finding meaning in meaningless |3| given the meaningless |4| in meaning making.
My main problem is	how my narccissim gets in the way of seeing the truth of the illusion of self.	|1| how my narccissim |2| gets in the way |3| of seeing the truth of the illusion of self.
These days, work	Is a rainbow of consciousness manifesting through each sparkling emanation called a 'me'.	|1| Is a rainbow of consciousness |2| manifesting through each sparkling emanation |3| called a 'me'.
Being with other people	Is a mutually mutating meeting of universes	|1| Is a mutually mutating meeting of universes
What I like to do best is	lay back and live the window that I have becomeas if a home with no walls, the infinite moves through as if there is no solid boundary.the veil is no more and the whole field that is everywhere exists where the inside and outside have no difference.	|1| lay back |2| and live the window |3| that I have become as if a home with no walls, |4| the infinite moves through |5| as if there is no solid boundary. |6| the veil is no more |7| and the whole field that is everywhere |8| exists where the inside and outside |9| have no difference.
Change is	the heartbeat of the universe... aliveness calling us forth!	|1| the heartbeat of the universe...|2| aliveness calling us forth!
When I get mad	i feel adrenaline through my human body and it is fierce and i love it because it moves me and it is no different than the Spirit which moves all.	|1| i feel adrenaline |2| through my human body |3| and it is fierce |4| and i love it |5| because it moves me |6| and it is no different |7| than the Spirit which moves all.
Raising a family	Captivates my whole being, just as every window on reality sees it is held in Love beyond knowing.	|1| Captivates |2| my whole being, |3| just as every window on reality sees |4| it is held in Love beyond knowing.
If I were in charge	Is an expression of the love animating all creation, the simple joy that moves all from rock to life to mind to All. It comes through this glorious One in all its parts and territories, from an inaccessible Heart, within, without.	|1| Is an expression |2| of the love animating all creation, |3| the simple joy that moves all from rock to life to mind to All. |4| It comes through this glorious One in all its parts and territories, from an inaccessible Heart, within, without.
Privacy	the human form consicousness separates from the One even for a moment.	|1| the human form |2| consicousness separates from the One even for a moment."""

Preprocess data

In [ ]:
# Accuracy in terms of clause count without subset removal while retaining case =  0.4897959183673469
# Accuracy in terms of clause count after subset removal while retaining case =  0.5510204081632653
# Accuracy in terms of chunk count without subset removal at lower case =  0.4897959183673469
# Accuracy in terms of chunk count after subset removal at lower ase =  0.5306122448979592


arr = [line.split("\t")[:-1] for line in tsv.split("\n")]
#arr = [[y.lower() for y in x ] for x in arr] 
expected_clause_splitup = [line.split("\t")[-1] for line in tsv.split("\n")]
expected_clause_splitup = [re.sub("\|\d+\|", ",,,", x).split(",,,")[1:] for x in expected_clause_splitup]
expected_clause_splitup = [[re.sub("\s+", ' ', word.strip()) for word in clause] for clause in expected_clause_splitup]
print(expected_clause_splitup[:5])

print(len(arr), len(arr[0]))
ips = []
for tok1, tok2 in arr:
    original_ip = "{} {}".format(tok1, tok2)
    ip = html.unescape(original_ip)
    ip = rgx.sub(' ', ip)
    ip = re.sub('\s+', ' ', ip)
    ips.append(ip)
    
print("Input Array Length", len(ips))

To build the clauses, get the subtrees of all verbs. For each child token, get their children recursively:
* IF the word is not a pronoun.
* If the child is not an [X]comp ( acomp, xcomp, ccomp)

In [48]:
COMPOSITE_NOUN_ATTRIBUTES = ["NOUN", "ADP", "ADJ", "PROPN"] #"VERB"
IGNORABLE_NOUN_DEPENDENCIES = ["acl"]
def get_substrings(doc, word):
    op = []
    if word.pos_ not in COMPOSITE_NOUN_ATTRIBUTES:
        return word.text
    
    for l in word.lefts:
        if l.dep_ not in IGNORABLE_NOUN_DEPENDENCIES:
            op.append(get_substrings(doc, l))
    op.append(word.text)
    for r in word.rights:
        if r.dep_ not in IGNORABLE_NOUN_DEPENDENCIES:
            op.append(get_substrings(doc, r))
    return ' '.join(op)

IGNORABLE_VERB_DEPENDENCIES = ["ccomp", "xcomp", "advcl", "conj"]
AUX_VERBS = ["am", "was", "are", "could", "would", "do", "has", "have", "can", "will", "did"]
def split_clauses(doc):
    op = []
    for word in doc:
        if word.pos_ == "VERB":
            clause_arr = []
            for l in word.lefts:
                if l.dep_ not in IGNORABLE_VERB_DEPENDENCIES:
                    clause_arr.append(get_substrings(doc, l))

            clause_arr.append(word.text) #TODO: Recurse ??
            
            for r in word.rights:
                if r.dep_ not in IGNORABLE_VERB_DEPENDENCIES:
                    clause_arr.append(get_substrings(doc, r))
                    
            if len(clause_arr) == 1 and clause_arr[0] in AUX_VERBS:
                continue
            op.append(" ".join(clause_arr))   
    return op

def split_text(doc):
    op = []
    for sent in doc.sents:
        tm = split_clauses(sent)
        op = op + tm
    return op

tokenized_sentences = []
for i in range(len(ips)):
    ip = ips[i]
    doc = nlp(ip)
    clauses = split_text(doc)
    
    a = displacy.render(doc, style='dep') # Saved in html folder

    html_str_arr = ["<html><body>"]
    html_str_arr.append("<div>Input: {}</div>".format(ips[i]))
    html_str_arr.append("<div>Expected Output: {}</div>".format(expected_clause_splitup[i]))
    html_str_arr.append("<div>Predicted Output: {}</div>".format(clauses))
    html_str_arr.append("<div>{}</div></body></html>".format(a))
    
    with open("html/file_{}.html".format(i), "w") as f:
        f.writelines(html_str_arr)
    print("##########################################################################3")
    print(i, ip)
    print(expected_clause_splitup[i])
    print(clauses)
    tokenized_sentences.append(clauses)
print("Html files edited with automated splitups")

##########################################################################3
0 sometimes i wish that my mom would come home more often.
['my mom', 'would come home more often.']
['sometimes i wish .', 'that my mom would come home often']
##########################################################################3
1 my family has a cat...i don't have a cat though i have 2 dogs
['has a cat...', "I don't have a cat", 'though I have 2 dogs']
['my family has a cat ...', "i do n't have a cat", 'though i have 2 dogs']
##########################################################################3
2 when i am nervous my hands get shaky
['my hands', 'get shaky']
['when i am nervous', 'my hands get shaky']
##########################################################################3
3 at times i worry about my own dreams nightmares i have thousands of them
['my own dreams, nightmares', 'I have thousands of them']
['at times i worry about my own dreams nightmares', 'i have thousands of them']
###########

##########################################################################3
28 at times i worry about how differently i see and process things from many of the people i encounter and whether i am wisely acting and interacting.
['how differently I see', 'and process things', 'from many of the people I encounter,', 'and whether I am wisely acting', 'and interacting.']
['at times i worry about see .', 'differently i see and and', 'process things from many of the people encounter', 'i encounter', 'whether i am wisely acting and', 'interacting']
##########################################################################3
29 what i like to do best is be a conduit. there is no i.
['be a conduit.', 'There is no I.']
['i like', 'what to do best', 'like is', 'is be a conduit .', 'there is no i.']
##########################################################################3
30 being with other people can be an isolating or a hugely enriching experience the latter when each others presence liberates 

##########################################################################3
43 what i like to do best is lay back and live the window that i have becomeas if a home with no walls the infinite moves through as if there is no solid boundary.the veil is no more and the whole field that is everywhere exists where the inside and outside have no difference.
['lay back', 'and live the window', 'that I have become as if a home with no walls,', 'the infinite moves through', 'as if there is no solid boundary.', 'the veil is no more', 'and the whole field that is everywhere', 'exists where the inside and outside', 'have no difference.']
['i like', 'what to do best', 'like is', 'is lay back and', 'live the window have', 'that i have becomeas', 'as if there is no solid boundary.the veil', 'is no more and the whole field is .', 'that is everywhere', 'exists', 'where the inside and outside have no difference']
##########################################################################3
44 change is th

Clauses cleanup:
1. Remove clauses which are subsets of another.
2. Remove the prompt text in resulting clauses

In [49]:
cleaned_clauses = []
for i in range(len(tokenized_sentences)):
    prompt = arr[i][0]
    clauses = tokenized_sentences[i]
    opclauses = clauses.copy()
    trav = 0
    while trav < len(clauses):
        idx = min(len(prompt), len(clauses[trav]))
        opclauses[trav] = clauses[trav][idx:]
        if idx == len(prompt):
            break
        trav += 1
        prompt = prompt[idx-1:]
    opclauses = [x.strip() for x in opclauses if x.strip() != ""]
    cleaned_clauses.append(opclauses)

print("Accuracy in terms of chunk count without subset removal = ", sum([len(expected_clause_splitup[i]) == len(cleaned_clauses[i]) for i in range(len(cleaned_clauses))])/len(cleaned_clauses))

cleaned_clauses_1 = []
for i in range(len(cleaned_clauses)):
    clauses = cleaned_clauses[i]
    ignore_clause = [True in [y.find(x)>=0 and x!= y for y in clauses] for x in clauses]
    new_clauses = [clauses[j] for j in range(len(clauses)) if not ignore_clause[j]]
    cleaned_clauses_1.append(new_clauses)

print("Accuracy in terms of chunk count after subset removal = ", sum([len(expected_clause_splitup[i]) == len(cleaned_clauses_1[i]) for i in range(len(cleaned_clauses_1))])/len(cleaned_clauses_1))
for i in range(len(cleaned_clauses_1)):
    print("######################################################################")
    print(i, len(expected_clause_splitup[i]), len(cleaned_clauses[i]), len(cleaned_clauses_1[i]), len(expected_clause_splitup[i]) == len(cleaned_clauses[i]), len(expected_clause_splitup[i]) == len(cleaned_clauses_1[i]), arr[i][0] + arr[i][1])
    print(expected_clause_splitup[i])
    print(cleaned_clauses[i])
    print(cleaned_clauses_1[i])


Accuracy in terms of chunk count without subset removal =  0.4897959183673469
Accuracy in terms of chunk count after subset removal =  0.5306122448979592
######################################################################
0 2 1 1 False False sometimes i wish thatmy mom would come home more often.
['my mom', 'would come home more often.']
['my mom would come home often']
['my mom would come home often']
######################################################################
1 3 3 3 True True my familyhas a cat...i don't have a cat though i have 2 dogs
['has a cat...', "I don't have a cat", 'though I have 2 dogs']
['has a cat ...', "i do n't have a cat", 'though i have 2 dogs']
['has a cat ...', "i do n't have a cat", 'though i have 2 dogs']
######################################################################
2 2 1 1 False False when i am nervousmy hands get shaky
['my hands', 'get shaky']
['my hands get shaky']
['my hands get shaky']
#################################################